In [ ]:
# This notebook requires py-pde in version 0.17.1 or later
# The package can be obtained from https://github.com/zwicker-group/py-pde
# Alternatively, it can be installed via pip or conda

import pde

# functools and _make_derivative2 are required for anisotropic laplacians
import functools
from pde.grids.operators.cartesian import _make_derivative2

# plotting functions
import matplotlib.pyplot as plt
import numpy as np

# fitting methods
import scipy

# a simple tracker to check parameter sweeps
import datetime
def ETA(step, maxStep, startTime):
    _ETA = None
    total_dt = 0
    dt = 0
    if step==0:
        _ETA = "Indeterminate"
    else:
        dt = datetime.datetime.now() - startTime
        dt = dt.seconds
        total_dt = dt/step * maxStep
        _ETA = (startTime + datetime.timedelta(seconds = total_dt))
        _ETA = str(_ETA.time())
        
    print(f"{int(100 * step / maxStep):>3} % completed. ETA: {_ETA} ({int(total_dt - dt)} seconds remain)." + '\t' * 5,
          end='\r')

# Reaction-Diffusion processes
Generic design features shared by the diverse biochemical interaction networks underlying protein pattern formation in cells include:
 - The dynamics (approximately) conserves the mass of each individual protein species: on the time scale of pattern formation neither protein production nor protein degradation are significant processes.
 - The biochemical reactions are characterised by (positive and negative) feedback mechanisms such that the chemical rate equations describing the dynamics of these reactions are generically nonlinear.
 - The proteins are typically transported by diffusive fluxes.
 
Then, the spatiotemporal dynamics of protein patterns is described by _mass-conserving reaction-diffusion_ (MCRD) equations.

The aspect of mass conservation is a constraint imposed by nature: in general, proteins do not appear out of nowhere, nor do they disappear into the void. For biological systems, the mass conservation plays a crucial role that will bother us later in the tutorial; for now, we will focus only on generic properties of pattern-forming reaction-diffusion systems. In the following, you will learn how to set up generic pattern-forming systems with a PDE solver, what individual parts of the differential equations mean, and how to interpret the results.

## The Fitzhugh-Nagumo model
We will analyse and implement a very generic system that shows stationary patterns here, namely the Fitzhugh-Nagumo model. This model was originally proposed as a simplified description of neuron spiking. The model features two components, $u(x,y,t)$ and $v(x,y,t)$, that interact nonlinearly with each other. The general formulation reads:
$$
\partial_t u(x,y,t) = D_u \, \nabla^2 u + f(u) - \sigma \cdot v \, , \\
\partial_t v(x,y,t) = D_v \, \nabla^2 v + u - v \, ,
$$
with $f(u)$ containing all the nonlinear interactions (see below).
For simplicity, the spatial/temporal variables $x$, $y$, and $t$ where omitted on the r.h.s. of the equations.
Note that these equations are a simplified version of the variants typically found in the literature, in particular regarding additional prefactors.

The nonlinear interaction term is the key component that leads to the formation of patterns in this model. Here, we will use
$$f(u) = - u^3 + \alpha \cdot u - \kappa \, .$$
In this representation, all parameters ($D_u$, $D_v$, $\alpha$, $\kappa$, and $\sigma$) are positive.

## Part 1: Analysing the problem analytically

### Problem 1: Understanding the differential equations
For each of the terms on the r.h.s. of the PDEs, state their effect on the dynamics of the system.
For each of the parameters, state how varying this parameter will qualitatively change the dynamics of the system.

_Hint: You should remain on a very broad level here, using phrases such as 'smoothens out rough profiles', 'leads to an increase/decrease of $v$ (or $u$)', etc._

_Hint: For examining the parameters, it helps to consider a specific state of the system. For example, if $u>0$ then the term $\alpha \cdot u$ will lead to further increase of $u$, but it will lead to a decrease if $u<0$._

### Solution 1
Terms in the equations:
- $D_u \nabla^2 u$: diffusion of quantity $u$
- $-u^3$: nonlinear decay of $u$ towards $u=0$
- $+\alpha \cdot u$: linear growth of $u$ away from $u=0$
- $-\kappa$: constant bias of $u$ dynamics to decrease $u$
- $-\sigma \cdot v$: decreases $u$ if $v$ is positive, and increases $u$ if $v$ is negative
- $D_v \nabla^2 v$: diffusion of quantity $v$
- $+u$: increases $v$ if $u$ is positive, decreases $v$ if $u$ is negative
- $-v$: linear decay of $u$ away from $u=0$

Parameters:
- $D_u$: diffusion constant of the activator species, smoothening out rough profiles in $u$
- $D_v$: diffusion constant of the inhibitor species, smoothening out rough profiles in $v$
- $\alpha$: for larger values of $\alpha$, the stable fixed points are further away from the unstable one, and a stronger perturbation is needed to overcome the diffusive smoothening
- $\kappa$: bias/skewness of the dynamics/fixed points. If $\kappa = 0$, then the system is perfectly symmetric with respect to sign changes. For $\kappa \neq 0$, the unstable fixed point is at $u \neq 0$. This decides whether the system shows a hole pattern or a spot pattern.
- $\sigma$ strength of the coupling to the quantity $v$: for $\sigma = 0$, the $u$ dynamics are independent of $v$, and no patterns will emerge. For $\sigma = \alpha = 1$, the system happens to be mass-conserving.

### Bonus Problem (Optional): Linear stability analysis and dispersion relation
The following problem requires some background in theoretical physics or mathematics. If you are comfortable with the terms used in the problem, or have time to spare after completing all other problems, you can try this bonus problem. It will teach you how the values for the constants used in the code were derived (since, of course, you can't just guess any values). You can even solve the problem without the suggested simplifications to obtain more general results.

Perform a linear stability analysis of the FitzHugh-Nagumo model for the case $\kappa = 0$ and $\sigma = 1$ (ignoring the diffusive component, i.e., assuming $D_u = D_v = 0$). What is the condition for the system to be linearly (un)stable?

_Hint: You will need to calculate the fixed points first. Then, determine whether the fixed point at $u^* = v^* = 0$ is stable or unstable._

Now calculate the dispersion relation, with the additional simplification that $\alpha = 1$. What are the conditions for the diffusion constants $D_u$ and $D_v$ to obtain a band of unstable modes?

_Hint: Use the following ansatz for calculating the dispersion relation to lowest order in $\delta u$ around the fixed point $u^* = v^* = 0$: $u(x,t) = \delta u \cdot \exp(\eta t) \sin(k \, x)$, and similar for $v$ It also helps to express one diffusion constant as a fraction of the other, e.g., $D_u = z \cdot D_v$._


### Bonus Problem: Solution
#### Fixed points:
In the well-mixed case, the equation for $v$ yields $u^* = v^*$ immediately. From the equation for $u$ it then follows that $u^*_1 = 0$, and $u^*_{2,3} = \pm \sqrt{\alpha - \sigma}$. The expression becomes more entangled for $\kappa \neq 0$.

#### Stability:
Still in the well-mixed case, one can write the equations linearized around $u^* = v^* = 0$ in matrix representation:
$$\partial_t \begin{pmatrix} \delta u \\ \delta v \end{pmatrix} = \begin{pmatrix} \alpha & -\sigma \\ 1 & -1 \end{pmatrix} \cdot \begin{pmatrix} \delta u \\ \delta v \end{pmatrix} \, .$$
The largest eigenvalue of this matrix determines the stability of the fixed point. The eigenvalues are
$$ \eta_\pm = \frac{\alpha - 1}{2} \bigl(1 \pm \sqrt{1 + 4 \frac{\alpha - \sigma}{(\alpha - 1)^2}}\bigr) \, .$$
For $\sigma = 1$, there is a positive eigenvalue (i.e., the fixed point is unstable) if $\alpha > 1$.

#### Dispersion relation:
The suggested simplification implies that the fixed point is now linearly (meta)stable, so the dispersion relation is bound to be zero at the mode $k=0$. Using the suggested ansatz, one can again write the system in matrix representation,
$$\partial_t \begin{pmatrix} \delta u \\ \delta v \end{pmatrix} = \begin{pmatrix} -D_u k^2 + \alpha & -\sigma \\ 1 & -D_v k^2 -1 \end{pmatrix} \cdot \begin{pmatrix} \delta u \\ \delta v \end{pmatrix} \, .$$
With $\alpha = \sigma = 1$, the dispersion relation then reads
$$ \eta(k) = \frac{1}{2} \biggl[-D_v k^2 (1+z) \pm \sqrt{D_v k^2 \cdot (1-z) \cdot \bigl(4 + D_v k^2 (1-z)\bigr)} \biggr] \, .$$
To check whether there is a band of unstable modes, it is sufficient to calculate the derivative of the dispersion relation w.r.t. k at $k \to 0$, since from the previous part we know that $\eta(k=0) = 0$. Alternatively, one may also just plot the obtained expression for varying values of $z$ and $D_v$. In the end, one finds that there is a band of unstable modes if $z < 1$.

## Part 2: Testing the system numerically
Different to the source-degradation, we don't know yet what to expect from the system. In particular, the size of the system and the grid resolution need to be determined without knowing the system dynamics. 

### Problem 2: Determining the numerical constraints
Use arbitrarily chosen values for the diffusion constants (e.g., $D_u = 0.1 \; [\mu m^2/s]$ and $D_v = 1.0 \; [\mu m^2/s]$) and the reaction coefficients ($\alpha = 1 \; [1/s]$, $\kappa = 0.1 \; [1/ (s \cdot \mu m ^2)]$, and $\sigma = 1 \; [1/s]$) to make an educated guess about the system size and spatial resolution.

### Solution 2
Using the same formula for the length scale as in the source-degradation problem ($\ell \sim \sqrt{D/k}$), we find that the characteristic length scale of the patterns should be on the order of $\sim \mu m$. To see a noticable pattern (i.e., not just a single blob), the system should be several tens of $\mu m$ large, while the spatial resolution should be below the $\mu m $ scale. Possible values are $L = 30 [\mu m]$ and `resolution` = 2, which means that individual grid cells are $0.5 [\mu m]$ in size.

In [ ]:
# Define the geometry of the system.
# This time, we use a square grid of width L. The mesh size can be adapted by changing `resolution`.
# Setting `resolution = 10`, for example, means that grid cells are 1/resolution = 0.1 (length units) in size.

L = 30            ## BLANK ##
resolution = 2    ## BLANK ##

# pde.CartesianGrid() creates a grid object.
# To avoid boundary effects, we use periodic boundary conditions here.

grid = pde.CartesianGrid(
    [[0, L], [0, L]],
    [int(resolution*L), int(resolution*L)],
    periodic = True
)
grid.plot();

In [ ]:
# Since we do not have any membrane dynamics, we do not need a special diffusion operator this time.

### Problem 3: Setting up the equations
According to your choice of the system size and resolution, it is now time to define the system parameters. Translate the equations of the Fitzhugh-Nagumo model into the required syntax of `py-pde`. As a reminder, the expression for $v$ is already given.

In [ ]:
D_u = 0.1
D_v = 1.0

alpha = 1.0
kappa = 0.1
sigma = 1.0

expr = {'u' : f'laplace(u) * {D_u} + {alpha} * u - u * u * u - {kappa} - {sigma} * v',  ## BLANK ##
        'v' : f'laplace(v) * {D_v} + u - v'}


Now you are almost ready to execute the simulation: only the initial conditions are still missing. In general, for pattern-forming systems, it is a good start to use an unstable fixed point with a sufficiently strong perturbation as initial states. In the Fitzhugh-Nagumo model, for $\kappa = 0$, one fixed point is at $u^* = v^* = 0$, and this fixed point is unstable _in a spatially extended system_ (in a 0D system, this need not be the case). Since we have a comparably small $\kappa$, the actual fixed point will be close to this, so for simplicity, we perturb the system around $(u, v) = (0,0)$.

`random_uniform(grid)` returns an array with values randomly chosen between 0 and 1. The code below transforms this to random values between -1 and 1, centered around the fixed point.

In [ ]:
# Construct the field objects
u = pde.ScalarField.random_uniform(grid) * 2. - 1.
v = pde.ScalarField.random_uniform(grid) * 2. - 1.


# Create the PDE object
eq = pde.PDE(expr)

# Set the initial values for all fields, and show it
field = pde.FieldCollection([u,v])
field.plot();

# Simulate the system and show the result
storage = pde.MemoryStorage()

res = eq.solve(field, t_range=200, tracker=[storage.tracker(1), "progress"])
res.plot();

## Part 3: Evaluation
### Problem 4: Interpreting the results
Look at the final state of the simulation. Do you observe a pattern? Do your results match the expactations? What is the order of magnitude of the pattern's characteristic length scale? Run the simulation again with different values for $\kappa$ and note down your observations.

### Solution 4
We can observe a spot pattern, where the distance between neighbouring spots is $\sim 10 \; [\mu m ]$.

Varying $\kappa$ changes the pattern from spots to holes.

### Problem 5: Quantifying the results
We now want to quantify the characteristic length scale. To do so, we will try to find the center of each spot in the pattern, and calculate the distances to the other spots. This analysis is done by the code below, which shows you a histogram of the spot distances, and returns a proxy for the mean distance between neighboring spots.

The function `get_length_scale` does not contain any blanks, however you are encouraged to walk through it and to understand the underlying idea. This will help you to properly interpret the results later on.

In [ ]:
def get_length_scale(data: np.array, show_stats: bool = True, show_centers: bool = False):
    # Make an empty array of the same size as the data array.
    # This is not necessary, but it helps to visualize things if needed, and is useful for debugging
    pattern_maxima = np.zeros_like(data)
    
    # Make a list to store all pattern spot maxima
    maxima_coordinates = []
    
    # Define some auxiliary quantities
    lx, ly = data.shape
    delta = 2 # size of the scanning window to find the maxima
    offsets = [(dx, dy) for dx in range(-delta, delta+1) for dy in range(-delta, delta+1)]
    offsets.remove((0,0)) # coordinate offsets that should be checked
    
    # Scan the entire array and check whether a pixel has the highes value of all neighbours
    for iy, ix in np.ndindex(data.shape):
        nbs = [data[(ix+dx)%lx, (iy+dy)%ly] for dx,dy in offsets]
        if data[ix,iy] > max(nbs):
            pattern_maxima[ix,iy] += 1.
            maxima_coordinates.append(np.array((ix,iy)))
            pattern_maxima[ix, iy] = 1.
            
    # Show the spot centers
    if show_centers:
        plt.imshow(data)
        plt.show();

        plt.imshow(pattern_maxima)
        plt.show();
            
    # Average area 'occupied' by a spot pattern: (total area)/(nr of spots)
    # Average distance between spots: sqrt((total area)/(nr of spots)):
    neighbour_distances = np.sqrt(L*L/len(maxima_coordinates))
    if show_stats:
        print(f"Nr. of spots: {len(maxima_coordinates)}")
        print(f"Occupied area per spot: {L*L/len(maxima_coordinates)}")
        print(f"Estimated average distance between spots: {neighbour_distances}")
    
    
    return np.mean(neighbour_distances)

In [ ]:
get_length_scale(storage.data[-1][0], show_centers = True)

### Problem 6: Sweeping
Again, we want to check what happens when we change one of the parameters. Here, we want to vary the diffusion constants. Sweep over an appropriate parameter regime and describe how the characteristic pattern length scale $\ell$ changes.

What (approximate) functional dependence of the length scale on the diffusion constant can you derive from your sweep? Is it meaningful to extrapolate your data? Where do you expect your results to fail?

In [ ]:
# Define a function that yields the characteristic length scale for a given parameter set
def simulate_and_get_length_scale(D_u = 0.1, D_v = 1.0, alpha = 1.0, kappa = 0.1, sigma = 1.0, tracker = "progress"):
    expr = {'u' : f'laplace(u) * {D_u} + {alpha} * u - u * u * u - {kappa} - {sigma} * v', 
            'v' : f'laplace(v) * {D_v} + u - v'}  
    
    # Construct the field objects
    u = pde.ScalarField.random_uniform(grid) * 2. - 1.
    v = pde.ScalarField.random_uniform(grid) * 2. - 1.

    eq = pde.PDE(expr)
    field = pde.FieldCollection([u,v])

    res = eq.solve(field, t_range=200, tracker=tracker)
    
    return get_length_scale(res.data[0], show_stats = False)    

In [ ]:
# Make a parameter sweep
# Empty list to store the results in
sweep_results = []

# Parameters to sweep over
sweep_parameters = np.power(10, np.arange(-.5, .5, 0.125))

# Nr of iterations per parameter to get a statistical average
sample_size = 4

# Launch the sweep
start_time = datetime.datetime.now()
for _ in sweep_parameters:
    tmp = []
    for __ in range(sample_size):
        ETA(list(sweep_parameters).index(_) * sample_size + __, len(sweep_parameters) * sample_size, start_time)
        tmp.append(simulate_and_get_length_scale(D_u = 0.1 * _, D_v = 1.0 * _, tracker = []))
    length_scale = np.mean(tmp)
    sweep_results.append(length_scale)

# Convert the results to a numpy array for better handling afterwards
sweep_results = np.array(sweep_results)

# Plot the data
plt.plot(sweep_parameters, sweep_results, 'o', label='data')
plt.title('Characteristic length scale vs. diffusion constant')
plt.xlabel('$D/D_0$')
plt.ylabel('Characteristic length $\ell$')
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Guessed function for the length scale depending on the sweep parameter
def fit_func(x, a, b):
    return a + np.sqrt(x * b)

# Fit parameters and covariance for the fit data
popt, pcov = scipy.optimize.curve_fit(fit_func, sweep_parameters, sweep_results)

# Visualization

plt.plot(sweep_parameters, sweep_results, 'o', label='data')
plt.plot(sweep_parameters, fit_func(sweep_parameters, *popt), 'g--',
         label='fit')
plt.title('Fit $\ell(D) = a + \sqrt{b \cdot D/D_0}$\nwith $D_0$ some reference diffusion constant:\n'+
         f'$a\;=${popt[0]: .3},\t$b\;=${popt[1]: .4}')
plt.xlabel('$D/D_0$')
plt.ylabel('Characteristic length $\ell$')
plt.legend(loc="upper left")
plt.show()

### Solution 6
From comparing the units, we know that the characteristic length scales as a square root function with the diffusion constant. Indeed, we can fit such a function to the data sweep.

The sweep has two obvious limitations: for too small diffusion constants (i.e., too small length scales), the spatial resolution of the numeric analysis is too low to properly identify the centers of spots, and thus no length scale can be extracted in these cases. On the other hand, once the characteristic length scale exceeds the system length, there will be only one spot in the system, so that again no characteristic length scale can be extracted.

In addition, in the limit $D_i \to 0$, the fit function fails, since no patterns can arise without diffusion. Extrapolation is therefore only meaningful in a limited parameter regime.

### Problem 7: (Non-)Linearities
The PDEs of the Fitzhugh-Nagumo model include a nonlinear term, namely $-u^3$. This nonlinearity is essential for the formation of patterns here. Test this explicitly by implementing a variant of the Fitzhugh-Nagumo model where the nonlinear term is replaced by $-u^3 \rightarrow -u^n$, with $n$ an integer exponent. In particular, test the cases $n \in \{1, 3, 5\}$, and describe your results. Give an explanation why you observe patterns in some cases, and why not in others.

In [ ]:
for n in [1, 3, 5]:

    D_u = 0.1 
    D_v = 1.0 

    alpha = 1.0
    kappa = 0.1
    sigma = 1.0 

    expr = {'u' : f'laplace(u) * {D_u} + {alpha} * u - u' + " * u" * (n-1) + f'- {kappa} - {sigma} * v',
            'v' : f'laplace(v) * {D_v} + u - v'}


    # Construct the field objects
    u = pde.ScalarField.random_uniform(grid) * 2. - 1.
    v = pde.ScalarField.random_uniform(grid) * 2. - 1.


    # Create the PDE object
    eq = pde.PDE(expr)

    # Set the initial values for all fields, and show it
    field = pde.FieldCollection([u,v])

    res = eq.solve(field, t_range=200, tracker=[])
    res.plot(title=f"n = {n}");

### Solution 7
For $n=1$, the system decays to an approximately homogeneous state. For $n=2$, the system is not bistable anymore, and apart from not making a pattern, it also diverges. For odd $n>1$, patterns are possible.

As a rule of thumb, the number of fixed points in a system is determined by the highest power in the PDEs. For $n=1$, there is only one fixed point. If the fixed point is stable, then the system converges to the fixed point, and if it is unstable, then the system diverges to $\pm \infty$, so no (stable) pattern is observed. For even orders, the system is not symmetric with respect to sign flips $u \to -u$, so that the system will diverge again. Only for odd orders larger than $1$, there are sufficiently many fixed points to prevent divergence while at the same time allowing for an unstable fixed point, so that in this case patterns are possible.

## Part 4: A variation of the system (Optional)
### Problem 8: Dynamic patterns
The pattern you analysed above is a stationary pattern, i.e. it will remain the same if you continue simulating for much longer times. For a different parameter regime, the Fitzhugh-Nagumo model shows dynamic patterns that reflect the 'neuron spiking' idea of the model. While these dynamic patterns are particularly difficult to analyse, they are at the same time very nice to look at. The code below simulates such a dynamic pattern. Run and enjoy!

In [ ]:
D_u = 0.01
D_v = 0.01

alpha = 2.0
kappa = 0.0
sigma = 2.0

expr = {'u' : f'laplace(u) * {D_u} + {alpha} * u - u * u * u - {kappa} - {sigma} * v',
        'v' : f'laplace(v) * {D_v} + u - v'}

# Construct the field objects
u = pde.ScalarField.random_uniform(grid) * 2. - 1.
v = pde.ScalarField.random_uniform(grid) * 2. - 1.


# Create the PDE object
eq = pde.PDE(expr)

# Set the initial values for all fields, and show it
field = pde.FieldCollection([u,v])
field.plot();

# Simulate the system for 100 time steps, and show the result
storage = pde.MemoryStorage()

res = eq.solve(field, t_range=100, dt = 0.01, tracker=[storage.tracker(0.2), "progress"])
res.plot();


In [ ]:
### Missing: animation of the results ###